# CMP263 - Aprendizagem de Máquina - INF/UFRGS

## Atividade Prática: Máquinas de Vetores de Suporte

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---



<br>

## **Aula 04** - **Tópico: Máquinas de Vetores de Suporte**

<br>

O algoritmo **Máquinas de Vetores de Suporte**, conhecido por **SVM** (de *Support Vector Machines*, em inglês), é amplamente utilizado para tarefas de classificação e regressão, possuindo uma grande versatilidade e alto poder de modelagem para problemas lineares e não-lineares.

Conforme discutido em aula, o algoritmo encontra a fronteira de decisão formulando a tarefa de aprendizado como um problema de otimização. O SVM localiza a reta ou hiperplano que separa as instâncias de treinamento em duas regiões de forma a maximizar a margem entre os vetores de suporte das duas classes analisadas. Para problemas multiclasse, a tarefa de classificação é decomposta em um conjunto de problemas de classificação binária.

A versatilidade do SVM vem do uso de diferentes tipos de kernel, que são funções que realizam transformações nos dados a fim de projetá-los em um espaço de dimensão superior, de forma a permitir que relações não-lineares sejam modeladas com um classificador linear. Além do kernel linear, outros tipos de kernel muito utilizados são o Polinomial e o Radial Basis Function (RBF).

<br>

**Objetivo deste notebook**: Exercitar o uso de SVM em tarefas de classificação, comparando diferentes tipos de kernel e realizando a otimização de hiperparâmetros do algoritmo.

<br>

---



##**Determinando a qualidade de vinhos verdes Portugueses**

*Seria possível prever as preferências humanas sobre vinhos verdes (principalmente de provadores profissionais) a partir de testes analíticos que quantificam propriedades químicas dos vinhos?*

Nesta atividade, vamos analisar um conjunto de observações sobre diversas amostras de vinho verde Português (do tipo branco), envolvendo suas propriedades químicas e classificação realizada por provadores através de análise sensorial.

O preço do vinho depende de um conceito bastante abstrato de apreciação do vinho a partir de análise sensorial pelos provadores, cuja opinião pode ter um alto grau de variabilidade. Outro fator chave na certificação e avaliação da qualidade do vinho são os testes análiticos realizados em laboratório e facilmente disponíveis na etapa de certificação. Estes testes levam em consideração fatores como acidez, nível de pH, presença de açúcar e outras propriedades químicas. Para o setor vitivinícola, seria de interesse que a análise de qualidade humana no processo de degustação pudesse estar relacionada com as propriedades químicas do vinho aferidas com os testes analíticos. Assim,  o processo de avaliação e certificação da garantia da qualidade do vinho poderia ser mais controlado.


O conjunto de dados a ser analisado nesta atividade possui informações para 4898 amostras de vinhos verdes (tipo branco), todas produzidos em uma determinada zona de Portugal. Os atributos são coletados para 12 propriedades diferentes dos vinho: uma das quais é Qualidade, com base em análise sensorial por provadores, e as restantes são propriedades químicas dos vinhos, incluindo densidade, acidez, teor alcoólico, etc. Todas as propriedades químicas dos vinhos são variáveis ​​contínuas. A qualidade é uma variável ordinal com uma classificação possível de 1 (pior) a 10 (melhor). Cada variedade de vinho é provada por três provadores independentes e a classificação final atribuída é a classificação mediana dada pelos provadores.

O conjunto de dados original foi publicado no artigo: *Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Modeling wine preferences by data mining from physicochemical properties. Decision support systems, 47(4), 547-553.*

O objetivo desta tarefa é abordar a questão apontada no início desta seção, desenvolvendo um modelo capaz de predizer a classificação de um vinho com base nas suas propriedades químicas avaliadas em testes analíticos.



---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
%matplotlib inline
import pandas as pd             # para análise de dados
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais
from sklearn.svm import SVC  ## para treinar um SVM
from sklearn.model_selection import train_test_split # para divisão de dados
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,ConfusionMatrixDisplay ## para avaliação dos modelos
sns.set()

In [ ]:
df = pd.read_table("https://drive.google.com/uc?export=view&id=11YsVJck74_gyADzGJSU9Uwn8cDq_l3BD",sep=";")
df.head()  # para visualizar apenas as 5 primeiras linhas

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'quality'* contém a classificação de qualidade de cada vinho, aferida pelos provadores. Vamos avaliar como as instâncias estão distribuídas entre os diferentes valores de *'quality'* presentes no dataset (originalmente os mesmos podem variar entre 0 e 10).

In [ ]:
## Distribuição do atributo alvo, 'quality'
sns.countplot(x='quality', data=df)

In [ ]:
## Imprimindo o valor exato de número de instâncias por nota
print(df.groupby('quality').size())

Podemos perceber que todos os vinhos no conjunto de dados estão classificados com notas entre 3 e 9. Entretanto, poucos vinhos estão classificados nos extremos da distribuição (notas 3 e 9).

Também é importante sempre confirmarmos os tipos de dados de cada atributo, bem como se existem valores faltantes. A célula seguinte faz esta inspeção.

In [ ]:
df.info()

## Para analisar valores faltantes, quando codificados como NaN, podemos usar o
## comando abaixo
df.isnull().sum()

Vamos analisar a distribuição de valores para os atributos, os quais são todos numéricos como foi possível confirmar na célula de código anterior.

In [ ]:
## Criando vetor com nome dos atributos
features_names = df.columns.drop(['quality'])

In [ ]:
## Gerar um gráfico para cada variável numérica com a distribuição
## de frequência. Avaliar a distribuição geral ou, opcionalmente,
## a distribuição por classe (classificação do vinho)

## Distribuição geral
def dist_plot(df,columns):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        g = sns.histplot(x=var, data=df)
    plt.tight_layout()

## Distribuição por classe
def dist_plot_perclass(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        sns.color_palette("pastel")
        g = sns.histplot(x=var, data=df, hue=label,palette='muted')
    plt.tight_layout()


dist_plot(df, features_names)
#dist_plot_perclass(df, features_names, 'quality')

**Pergunta >>>** Neste ponto já é possível avaliar a necessidade de normalização dos dados para treinamento do SVM. Analise o quanto os dados demandam pré-processamento para normalização dos valores.

> ***Sua resposta aqui:***

O gráfico abaixo mostra de outra forma como variam os valores dos atributos, considerando todas as instâncias no conjunto de dados.

**Pergunta >>>** Quais atributos possuem as escalas de valores mais discrepantes em relação ao conjunto total de atributos?

> ***Sua resposta aqui:***

In [ ]:
df.drop(['quality'],axis=1).plot(figsize=(15,7))

Por fim, para auxiliar no entendimento do problema, é interessante avaliar a correlação entre atributos, bem como a correlação dos atributos com a classificação da qualidade do vinho. Vamos criar um heatmap com a correlação de Pearson entre os atributos da base.

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), annot=True, cmap = 'coolwarm')
plt.show()

É possível visualizar o comportamente da correlação entre atributos e a classe através de lineplots. Visualize os dados para alguns atributos selecionados que apresentem correlação positiva ou negativa com quality (mesmo que a correlação seja fraca). Digite o nome do atributo no campo abaixo (feature_sel).

In [ ]:
feature_sel = "density" #@param {type:"string"}

plt.figure(figsize=(10,5))
sns.lineplot(data=df, x="quality",y=feature_sel,color='r')

---


### Estruturando a tarefa de classificação: vinho bom ou medíocre?

A análise exploratória dos dados nos mostrou que a qualidade dos vinhos no conjunto de dados disponível varia entre 3 e 9, com poucas instâncias sendo classificadas com as notas extremas. A distribuição dos dados sugere que a grande maioria dos vinhos possuem notas 5, 6 e 7. A falta de representatividade de vinhos com as demais classificações pode se apresentar como um problema para o desenvolvimento do modelo.

**Pergunta >>>** Qual possivelmente será o desempenho de um modelo treinado com estes dados para a classificação de novos vinhos com nota 9-10 ou nota 0-2?


> ***Sua resposta aqui:***

Para dar continuidade à modelagem do problema de classificação dos vinhos, vamos **investigar a possibilidade de distinguir bons vinhos de vinhos medíocres ou ruins**, utilizando **a nota de 7** como um ponto de corte. Vinhos com nota 7 ou superior serão classificados como bons vinhos (1), enquanto vinhos com notas 0-6 serão classificados como medíocres (0).


In [ ]:
df['quality'] = df['quality'].replace([3, 4, 5, 6], 0)
df['quality'] = df['quality'].replace([7, 8, 9], 1)
sns.countplot(x='quality', data=df)


---


### Criando conjuntos de treino, validação e teste

Faremos a divisão dos dados em treino, validação e teste na proporção (sugerida) de 70%/15%/15% para fazer avaliação de desempenho dos modelos com um *holdout de 3 vias*. Na célula de código abaixo, os dados são inicialmente divididos entre duas variáveis: uma variável contendo os atributos e outra a classe a ser predita. Na sequência, são implementados os seguintes passos:


1.   Utilizada a função train_test_split() para dividir os dados em treino, validação e teste.
2.   Normalização  dos dados usando o método MaxMin. Lembre-se que primeiramente os parâmetros de normalização são estimados a partir dos dados de treino e então aplicados a todos os três conjuntos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['quality'], axis=1)
y = df['quality'].values

In [ ]:
## Definindo as proporções de treino, validação e teste.
train_ratio = 0.70
test_ratio = 0.15
validation_ratio = 0.15

## Fazendo a primeira divisão, para separar um conjunto de teste dos demais.
## Assuma X_train e y_train para os dados de treinamento e X_test e y_test para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio,stratify=y,random_state=42)

## Fazendo a segunda divisão, para gerar o conjunto de treino e validação a partir
## do conjunto de 'treinamento' da divisão anterior
## Assuma X_train e y_train para os dados de treinamento e X_valid e y_valid para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio),stratify=y_train,random_state=42)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
## O MinMaxScaler transformará os dados para que fiquem no intervalo [0,1]
scaler = MinMaxScaler() #StandardScaler()

## Iniciar a normalização dos dados. Primeiro fazer um 'fit' do scaler nos
## dados de treino. Esta etapa visa "aprender" os parâmetros para normalização.
## No caso do MinMaxScales, são os valores mínimos e máximos de cada atributo
scaler.fit(X_train)

## Aplicar a normalização nos três conjuntos de dados:
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

Com os dados normalizados, podemos melhor inspecionar a distribuição de valores por classe, considerando o problema de classificação binário. Vamos fazer a análise para o conjunto de treino, observando as medianas para cada classe.

In [ ]:
df_train_norm =  pd.DataFrame(X_train,columns=features_names)
df_ytrain =  pd.DataFrame(y_train,columns=['quality'],dtype=int)
df_train_norm =  pd.concat([df_train_norm,df_ytrain], axis=1)
ave_values = df_train_norm.groupby("quality").median()
ave_values.plot(kind="bar",figsize=(15,7))

---


### Treinamento de modelos SVM com Kernel Linear



Como vimos em aula, o modelo SVM com Kernel Linear é o "mais simples" dentre os modelos de SVM. Ele possui o hiperparâmetro C (termo de regularização), comum a todos os modelos de SVM, e que precisa ser otimizado para cada problema.

#### Otimização "manual" do hiperparâmetro C
No código abaixo, vamos fazer a otimização do hiperparâmetro C através da implementação de um loop. Para cada modelo, vamos avaliar a acurácia, recall e precisão. Vamos assumir que o melhor modelo será aquele que maximiza a acurácia.

In [ ]:
## Definindo um array para armazenar o desempenho de cada modelo treinado e avaliado
perf_valid = []

## Definindo valores de C a serem testados
param_grid_C = [0.1, 1, 5, 10, 50, 100] #valores para C, termo de regularização

# Treinando e avaliado os modelos com cada valor de hiperparâmetro especificado
for ii in range(len(param_grid_C)):
    clf = SVC(kernel='linear',C=param_grid_C[ii],random_state=42, class_weight='balanced') ##class_weight minimiza o efeito de termos mais exemplos para vinhos medíocres/ruins
    clf.fit(X_train, y_train)
    pred_i = clf.predict(X_valid)
    perf_valid.append([param_grid_C[ii],accuracy_score(y_valid, pred_i),recall_score(y_valid, pred_i),precision_score(y_valid, pred_i)])

In [ ]:
perf_df = pd.DataFrame(perf_valid, columns=['C','accuracy','recall','precision'])
perf_df

In [ ]:
plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df, id_vars=['C'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='C',y='value',hue='variable',palette='muted',marker='o')

**Pergunta >>>** Qual valor de C maximiza a acurácia? Como são os valores de acurácia, recall e precisão para este modelo?

> ***Sua resposta aqui:***

**Pergunta >>>** Qual o impacto de usar a opção `class_weight='balanced'` no código? Tente executar sem esta opção e observe o efeito nos resultados.

> ***Sua resposta aqui:***

#### Otimização do hiperparâmetro C com GridSearchCV

O scikit-learn disponibiliza o método GridSearchCV, que permite fazer a otimização de hiperparâmetros de forma mais automática. É especialmente útil quando queremos testar c**ombinações** de valores para diferentes hiperparâmetros. O código abaixo aplica o GridSearchCV usando as divisões de dados de treino e validação pré-definidas (através do método PredefinedSplit). Portanto, devemos chegar ao mesmo resultado da análise anterior.

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV ## para auxiliar na otimização de hiperparâmetros

# Cria lista com os dados de treinamento com índice -1 e dados de validação com índice 0
# Concatena os dados de treino e validação com as partições pré-definidas
split_index = [-1]*len(X_train) + [0]*len(X_valid)
X_gridSearch = np.concatenate((X_train, X_valid), axis=0)
y_gridSearch = np.concatenate((y_train, y_valid), axis=0)
pds = PredefinedSplit(test_fold = split_index)

## Define métricas de desempenho a serem estimadas
scoring = {'Accuracy':'accuracy', 'Precision': 'precision', 'Recall':'recall'}

## Define o algoritmo base da otimização de hiperparâmetros
estimator = SVC(kernel='linear',class_weight='balanced')

## Define a grid de hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 5, 10, 50]}
#param_grid = {'C': [0.1, 1, 5, 10, 50, 100]}#, 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
#param_grid = {'C': [0.1, 1, 5, 10, 50, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}


## Aplica GridSearch com as partições de treino/validação pré-definidas
gridS = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid,
                   scoring=scoring,
                   refit='Accuracy', ##métrica a ser utilizada para definir o melhor modelo, retreinando-o com toda a base
                   return_train_score=True)
gridS.fit(X_gridSearch, y_gridSearch)
print('Desempenho máximo obtido com: {}'.format(gridS.best_params_))

In [ ]:
print(gridS.cv_results_)

A visualização dos dados pode nos auxiliar a explorar o resultado da otimização de hiperparâmetros. O código abaixo criar um gráfico a partir dos resultados do GridSearchSV, focando na variação de um hiperparâmetro. Neste caso, analisamos C, o único hiperparâmetro variado na análise. (*OBS.: Não se preocupe se não entender todos os detalhes de implementação da função, ela é apenas um utilitário na visualização de dados*)

In [ ]:
## O código desta célula cria um gráfico de variação de desempenho de acordo com
## o valor do hiperparâmetro C.

results = gridS.cv_results_

plt.figure(figsize=(10, 7))
plt.title("Resultados do GridSearchCV",
      fontsize=16)

plt.xlabel("Hyperparameter") ##nome do parâmetro a ser analisado
plt.ylabel("Performance")

ax = plt.gca()

## Criar um numpy array para os resultados do hiperparâmetro a ser analisado.
## O hiperparâmetro C está identificado no objeto retornado pelo gridSearchCV
## como param_C
X_axis = np.array(results['param_C'].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k', 'b', 'r']):
    for sample, style in (('train', '--'), ('test', '-')):
       sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
       sample_score_std = results['std_%s_%s' % (sample, scorer)]
       ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == 'test' else 0, color=color)
       ax.plot(X_axis, sample_score_mean, style, color=color,
            alpha=1 if sample == 'test' else 0.7,
            label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    ## Plota uma linha vertical para o valor de hiperparâmetro que maximiza a métrica de desempenho
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
        linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    ## Anota o valor do melhor score
    ax.annotate("%0.3f" % best_score,
            (X_axis[best_index], best_score + 0.008))

plt.legend(loc="best")
plt.grid(False)
plt.show()

Analisando o desempenho do modelo treinado com o melhor valor de C nos dados de teste.

In [ ]:
y_pred_svmLinear = gridS.predict(X_test) ##predição usando SVM com a melhor configuração de hiperparâmetros encontrada

In [ ]:
cm = confusion_matrix(y_test, y_pred_svmLinear,labels=gridS.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gridS.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_pred_svmLinear),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_pred_svmLinear,pos_label=1),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_pred_svmLinear,pos_label=1),3)))

---


### Comparando diferentes funções de Kernel no SVM


Nesta seção, vamos comparar diferentes tipos de Kernel em um algoritmo SVM. Utilizaremos o método GridSearchCV do scikit-learn pela praticidade do mesmo. Entretanto, também seria possível realizar a mesma análise com um loop definido por nós, testando diferentes combinações de valores de hiperparâmetros.

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV ## para auxiliar na otimização de hiperparâmetros


## Cria lista com os dados de treinamento com índice -1 e dados de validação com índice 0
## Concatena os dados de treino e validação com as partições pré-definidas
split_index = [-1]*len(X_train) + [0]*len(X_valid)
X_gridSearch = np.concatenate((X_train, X_valid), axis=0)
y_gridSearch = np.concatenate((y_train, y_valid), axis=0)
pds = PredefinedSplit(test_fold = split_index)

# ## Define métricas de desempenho a serem estimadas
scoring = {'Accuracy':'accuracy', 'Precision': 'precision', 'Recall':'recall'}

## Define o algoritmo base da otimização de hiperparâmetros
estimator = SVC(class_weight='balanced')

## Define a grid de hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 5, 10, 50, 100], 'kernel': ['rbf', 'poly', 'sigmoid']}#,'gamma': [1,0.1,0.01,0.001]} ## gamma foi removido para reduzir tempo de execução

## Aplica GridSearch com as partições de treino/validação pré-definidas
gridS2 = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid,
                   scoring=scoring,
                   refit='Accuracy', ##métrica a ser utilizada para definir o melhor modelo, retreinando-o com toda a base
                   return_train_score=True)
gridS2.fit(X_gridSearch, y_gridSearch)
print('Desempenho máximo obtido com: {}'.format(gridS2.best_params_))

In [ ]:
results = gridS2.cv_results_

plt.figure(figsize=(10, 7))
plt.title("Resultados do GridSearchCV",
      fontsize=16)

plt.xlabel("Hyperparameter") ##nome do parâmetro a ser analisado
plt.ylabel("Performance")

ax = plt.gca()

## Criar um numpy array para os resultados do hiperparâmetro a ser analisado
X_axis = np.array(results['param_kernel'].data)#, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k', 'b', 'r']):
    for sample, style in (('train', '--'), ('test', '-')):
       sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
       sample_score_std = results['std_%s_%s' % (sample, scorer)]
       ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == 'test' else 0, color=color)
       ax.plot(X_axis, sample_score_mean, style, color=color,
            alpha=1 if sample == 'test' else 0.7,
            label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    ## Plota uma linha vertical para o valor de hiperparâmetro que maximiza a métrica de desempenho
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
        linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    ## Anota o valor do melhor score
    ax.annotate("%0.3f" % best_score,
            (X_axis[best_index], best_score + 0.008))

plt.legend(loc="best")
plt.grid(False)
plt.show()